# 集成学习

集成学习算法的主要思想是，根据训练集数据训练出若干个个体学习器，通过一定的结合策略，把这些单个的个体学习器结合形成一个强学习器。所以集成学习的关键有两点：
1. 如何构建具有差异性的分类器；
2. 如何多这些分类器的结果进行整合。

### 个体学习器的生成

个体学习器可以是相同的，比如说都是决策树模型，也可以是不同的，比如对训练集采用支持向量机个体学习器，逻辑回归个体学习器和朴素贝叶斯个体学习器来学习，再通过某种结合策略来确定最终的分类强学习器。

根据个体学习器之间是否存在强依赖关系，可以生成分为两个类别，一类是Bagging，一类是Boosting.

##### Bagging
Bagging 算法中弱学习器之间没有依赖关系，可以并行生成，具体步骤是，在原始样本中随机抽样获取子集，用随机抽样的子集训练基学习器（base_estimator），因为训练集数据不同，即使是相同类型的个体学习器训练出来的模型也会具有差异性，然后对每个基学习器的结果求平均，最终得到的预测值。随机获取样本子集的方法有很多中，最常用的是有放回抽样的booststrap，也可以是不放回的抽样。

#### Boosting
Boosting算法的工作机制是首先从训练集用初始权重训练出一个弱学习器1，根据弱学习的学习误差率表现来更新训练样本的权重，使得之前弱学习器1学习误差率高的训练样本点的权重变高，使得这些误差率高的点在后面的弱学习器2中得到更多的重视。然后基于调整权重后的训练集来训练弱学习器2.，如此重复进行，直到弱学习器数达到事先指定的数目T，最终将这T个弱学习器通过集合策略进行整合，得到最终的强学习器。

常用的有 AdaBoost 和 GBDT

### 分类器的结合策略(对基分类器结果进行整合)
##### 回归
1. 简单平均:就是取各个学习器结果的平均值.
2. 加权平均:加权平均, 每个学习器都有权重.

##### 分类
1. 简单投票：就是每个分类器的权重大小一样，少数服从多数，类别得票数超过一半的作为分类结果
2. 加权投票：每个分类器权重不一。
3. 概率投票：有的分类器的输出是有概率信息的，因此可用概率投票。

# sklearn 中的 Bagging

In [7]:
from sklearn.datasets import make_moons
from sklearn.ensemble import BaggingClassifier # BaggingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
# 1.获取数据
X, y = make_moons(n_samples=200)
# 2. 划分训练集和测试集
X_tr,X_te,y_tr,y_te = train_test_split(X, y)
# 3.构造模型
#训练一个100个基分类器为决策树的集成，每一个基分类器都是在数据集上有放回采样100个训练实例下进行训练
# bootstrap=True 表示有放回的采样
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=10, bootstrap=True, max_samples=100)
# 4.训练模型
bag_clf.fit(X_tr, y_tr)
# 5.预测
y_pred = bag_clf.predict(X_te)

print(accuracy_score(y_pred, y_te))

1.0


集成的分类比起单一决策树的分类产生情况更好：集成有一个可比较的偏差但是有一个较小的方差,另外，若采用的是有放回的采样，那么每一个分类器都会有37%的数据没有被采样参与训练，这些样本实例叫做Out-of-Bag实例,因为在训练中分类器从来没有看到过oob实例，所以它可以在这些实例上进行评估，而不需要单独的验证集或交叉验证。可以拿出每一个分类器的oob来评估集成本身

In [9]:
# 只需要加上参数 oob_score=True即可让分类器进行自我评估
bag_clf = BaggingClassifier(DecisionTreeClassifier(), oob_score=True, n_estimators=10, bootstrap=True, max_samples=100)
bag_clf.fit(X_tr, y_tr)
print("oob_score: ", bag_clf.oob_score_) # 0.94代表可以再测试集上达到94%的准确率
y_pred = bag_clf.predict(X_te)
print(accuracy_score(y_pred, y_te))

oob_score:  0.94
1.0


In [10]:
# 随机森林，在sklearn中可以认为是优化了的Bagging
from sklearn.ensemble import RandomForestClassifier

# 在构造时可使用DecisionTreeClassifier的所有超参数（决定数怎么生长）
# 也可使用BaggingClassifierd的超参数加起来来控制集成本身
rnd_clf = RandomForestClassifier(n_estimators=20, max_leaf_nodes=16)

# 相当于如下形式的 Bagging
# bag_clf=BaggingClassifier(DecisionTreeClassifier(splitter="random",max_leaf_nodes=16),n_estimators=500,max_samples=1.0,bootstrap=True,n_jobs=-1)
rnd_clf.fit(X_tr, y_tr)
y_pred = bag_clf.predict(X_te)
print(accuracy_score(y_pred, y_te))

1.0


### 特征重要度
单一决策树，重要的特征会出现在更靠近根部的位置，而不重要的特征会经常出现在靠近叶子的位置。因此可以通过计算一个特征在森林的全部树中出现的平均深度来预测特征的重要性

In [11]:
# sklearn在训练后会自动计算每个特征的重要度。可以通过feature_importances_变量来查看结果
# 这里以 iris 数据为例

from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target

rnd_clf = RandomForestClassifier(n_estimators=200)
rnd_clf.fit(X, y)

for f, i in zip(iris['feature_names'], rnd_clf.feature_importances_):
    print(f, i)

sepal length (cm) 0.09851716283166637
sepal width (cm) 0.03001321804260488
petal length (cm) 0.46655388856744584
petal width (cm) 0.40491573055828306


# sklearn中的Boosting

## Adaboost
AdaBoost 是一种Boosting方法，与Bagging不同的是，Adaboost中不同的子模型必须是串行训练获得的，每个新的子模型都是根据 已训练出的模型 来进行训练的。AdaBoost中每个训练样本都有一个权重，这些权重构成了一个向量W，初始值都为w_i = 1/N。Adaboost中每次迭代生成新的子模型使用的训练数据都相同，但是样本的权重会不一样。AdaBoost会根据当前的错误率，增大错误样本权重，减小正确样本权重的原则更新每个样本的权重。不断重复训练和调整权重，直到训练错误率或弱学习器的个数满足用户指定的值为止。Adaboost的最终结果为每个弱学习器加权的结果。使用sklearn中的Adaboot时，主要调节的参数有n_estimator(多少棵树)、max_depth（每棵树的深度。复杂度）或者min_samples_leaf(最少的叶子节点)。

In [12]:
from sklearn.ensemble import AdaBoostClassifier

ad_clf = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=20, 
                            learning_rate=0.5, algorithm='SAMME.R')

ad_clf.fit(X_tr, y_tr)
y_pred = bag_clf.predict(X_te)
print(accuracy_score(y_pred, y_te))

1.0


## GBDT
GDBT也是一种Boosting方法，每个子模型是根据已训练出的学习器的（残差）训练出来的，子模型是串行训练获得，不易并行化。GDBT使用非常广泛的，能分类，能回归预测。GDBT结合了梯度迭代和回归树，准确率非常高，但是也有过拟合的风险。GDBT中迭代的残差的梯度，残差就是目前结合所有得到的训练器预测的结果与实际值的差值，GDBT的使用也非常的简单，主要调节的参数有确定需要多少棵树（n_estimator）、每棵树的复杂度（max_depth，max_leaf_node）、损失函数(loss)以及学习率(learning_rating)。为了防止过拟合一般学习率会选小一点的(<0.1)，learning_rate会影响n_estimator，需要权衡，选择最佳的组合参数。

In [23]:
# 用DecisionTreeRegressor去拟合训练集（例如一个有噪音的训练集）
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
X = np.random.rand(200, 1) - 1 
y = 3*X**2 + 0.05 * np.random.randn(200,1) 
X, y = X.reshape((-1, 1)), y.reshape((-1, 1))
# 构造第一个学习器
clf_1 = DecisionTreeRegressor(max_depth=2)
clf_1.fit(X, y)

# 利用第一个分类器的残差来训练第二个学习器
y2 = y - clf_1.predict(X)
clf_2 = DecisionTreeRegressor(max_depth=2)
clf_2.fit(X, y2)

# 利用第二个分类器的残差来训练第三个学习器
y3 = y2 - clf_2.predict(X)
clf_3 = DecisionTreeRegressor(max_depth=2)
clf_3.fit(X, y3)

# 现在有了一个包含三个回归器的集成。它可以通过集成所有树的预测来在一个新的实例上进行预测。
y_pred = sum(clf.predict([[0.5]]) for clf in [clf_1, clf_2, clf_3])
#print(mean_squared_error(y_pred, y[1]))
#print(y_pred)
print(y_pred)

[[ 0.01035394  0.01035394  0.01035394 -1.49874798  0.01035394  0.01035394
  -0.6539953  -0.6539953  -1.49874798 -2.3990125   0.01035394  0.01035394
  -1.49874798  0.01035394 -1.49874798 -1.49874798 -0.6539953  -0.6539953
  -2.3990125  -1.49874798 -0.6539953   0.01035394  0.01035394 -1.49874798
  -0.6539953   0.01035394 -2.3990125  -0.6539953  -0.6539953   0.01035394
  -0.6539953  -0.6539953  -0.6539953  -2.3990125   0.01035394 -1.49874798
  -1.49874798 -1.49874798 -2.3990125  -1.49874798 -2.3990125  -0.6539953
  -1.49874798  0.01035394  0.01035394 -1.49874798 -0.6539953  -0.6539953
   0.01035394  0.01035394  0.01035394 -1.49874798  0.01035394 -1.49874798
  -1.49874798 -2.3990125  -2.3990125  -2.3990125  -2.3990125  -2.3990125
   0.01035394 -0.6539953   0.01035394 -1.49874798 -0.6539953  -0.6539953
  -0.6539953   0.01035394  0.01035394  0.01035394 -1.49874798  0.01035394
   0.01035394  0.01035394 -0.6539953  -0.6539953  -1.49874798  0.01035394
   0.01035394  0.01035394  0.01035394 -1.49

In [27]:
from sklearn.ensemble import GradientBoostingRegressor

gbdt_clf = GradientBoostingRegressor(n_estimators=3, max_depth=2, learning_rate=.1)

gbdt_clf.fit(X, y)
gbdt_clf.predict(0.5)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0.83234355])

超参数learning_rate确立了每个树的贡献。如果你把它设置为一个很小的树，例如0.1，在集成中就需要更多的树去拟合训练集，但预测通常会更好。这个正则化技术叫做shrinkage。为了找到树的最优数量，你可以使用早停技术（。最简单使用这个技术的方法就是使用staged_predict()：它在训练的每个阶段（用一棵树，两棵树等）返回一个迭代器

In [28]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y)
gbrt = GradientBoostingRegressor(n_estimators=100, max_depth=2)
gbrt.fit(X_tr, y_tr)

errors = [mean_squared_error(y_pred, y_te) for y_pred in gbrt.staged_predict(X_te)]

best_n_estimators = np.argmin(errors)
gbrt = GradientBoostingRegressor(n_estimators=best_n_estimators, max_depth=2)
gbrt.fit(X_tr, y_tr)
y_pred = gbrt.predict(X_te)
print(mean_squared_error(y_pred ,y_te))

0.004264256478439723


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
